Importing correct header files and functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing data into pandas dataframe

In [2]:
Rock_file="ClassicRock_3Ho3iO0iJykgEQNbjB2sic.csv"
HipHop_file="HipHopClassics_5CRJExGSQFYXem2hT5GFkd.csv"
HeavyMetal_file="HeavyMetalClassics_27gN69ebwiJRtXEboL12Ih.csv"
Dance_file="DanceHits_5oKz4DsTP8zbL97UIPbqp4.csv"
Jazz_file="JazzClassic_37i9dQZF1DXbITWG1ZJKYt.csv"
Kpop_file="KpopClassics_0jB4ANR4ox65etDMnxvGLp.csv"
Classical_file="ClassicalStudy_6wObnEPQ63a4kei1sEcMdH.csv"
Country_file="CountryHits_1mJhSx6aYQmINsZ8dG4gzU.csv"

Taking necessary columns of file

In [7]:
#Sticking with just the basic parameters for now
parameter_columns = [1,2,4,5,7,8,9]

#Saving these other ones for later
# key_column=3
# tempo_column=11
# duration_column=17


#Testing to make sure read column numbers correctly
data = pd.read_csv(Rock_file, usecols=parameter_columns) 
data.head()

,danceability,energy,loudness,mode,acousticness,instrumentalness,liveness
0,0.310,0.700,-5.678,1,0.01100,0.009650,0.0828
1,0.671,0.373,-18.064,1,0.25700,0.000080,0.0481
2,0.579,0.508,-9.484,1,0.00574,0.000494,0.0575
3,0.508,0.834,-6.205,1,0.01830,0.000000,0.0732
4,0.596,0.605,-12.145,1,0.18100,0.000331,0.0863


Extra Column to hold the playlist number

In [9]:
print(len(data))
playlist_number=1
playlist_numbers = playlist_number*np.ones(len(data))
data["playlist_number"]=playlist_numbers
data.head()

300


,danceability,energy,loudness,mode,acousticness,instrumentalness,liveness,playlist_number
0,0.310,0.700,-5.678,1,0.01100,0.009650,0.0828,1.0
1,0.671,0.373,-18.064,1,0.25700,0.000080,0.0481,1.0
2,0.579,0.508,-9.484,1,0.00574,0.000494,0.0575,1.0
3,0.508,0.834,-6.205,1,0.01830,0.000000,0.0732,1.0
4,0.596,0.605,-12.145,1,0.18100,0.000331,0.0863,1.0


Putting 200 songs from each playlist into a single dataframe

In [60]:
file_names=[Rock_file, HeavyMetal_file, Dance_file, Jazz_file, Kpop_file, Classical_file]#, HipHop_file]#, Country_file]
parameter_columns = [1,2,4,5,7,8,9]

# Most convenient to set up dataframe with initial file, then systematically loop through the others
#Sticking with just the basic parameters for now
parameter_columns = [1,2,5,6,7,8,9]#,19 #to include song name
nrows=200



#1 indexing here more natural
playlist_number=1
data = pd.read_csv(file_names[playlist_number-1], usecols=parameter_columns, nrows=nrows)

playlist_numbers = playlist_number*np.ones(len(data))
data["playlist_number"]=playlist_numbers #Now has extra playlist column



# Now looping through the remaining filenames
for playlist_number in range(2,len(file_names)+1):
    # dtmp := Temporary dataframe, will append to df.data once data filtered and prepped
    dtmp= pd.read_csv(file_names[playlist_number-1], usecols=parameter_columns, nrows=nrows)

    playlist_numbers = playlist_number*np.ones(len(dtmp)) # array of playlist_number
    dtmp["playlist_number"]=playlist_numbers #Now has extra playlist column
    data=data.append(dtmp, ignore_index=True) #Append back to main df.data
    
display(data)

,danceability,energy,mode,speechiness,acousticness,instrumentalness,liveness,playlist_number
0,0.310,0.7000,1,0.0470,0.01100,0.009650,0.0828,1.0
1,0.671,0.3730,1,0.0323,0.25700,0.000080,0.0481,1.0
2,0.579,0.5080,1,0.0270,0.00574,0.000494,0.0575,1.0
3,0.508,0.8340,1,0.0386,0.01830,0.000000,0.0732,1.0
4,0.596,0.6050,1,0.0255,0.18100,0.000331,0.0863,1.0
...,...,...,...,...,...,...,...,...
1195,0.529,0.0481,1,0.0596,0.99200,0.907000,0.1050,6.0
1196,0.458,0.1190,1,0.0542,0.98900,0.910000,0.0935,6.0
1197,0.258,0.0192,1,0.0533,0.96600,0.928000,0.1040,6.0
1198,0.423,0.1270,1,0.0336,0.98500,0.844000,0.1090,6.0


Checking for duplicate tracks

In [61]:
print(data.duplicated().sum()) #prints number of duplicates
print(data.loc[data.duplicated(),:]) #prints column of duplicate

0
Empty DataFrame
Columns: [danceability, energy, mode, speechiness, acousticness, instrumentalness, liveness, playlist_number]
Index: []


Simple model which calculates average of each parameter for every playlist and minimises the cosine simularity between this and a new song.

In [87]:
#Calculating average values for each playlist and storing in an array
AverageVals = []
for i in range(1,len(file_names)+1):
    AverageVals.append(np.array(data.loc[data["playlist_number"] == float(i)].sum()/200))
    #print(data.loc[data["playlist_number"] == float(i)].sum()/200)
print(AverageVals[0][7])

1.0


Consine Simularity

In [82]:
def CosSimularity(X,Y):
    XY = Xabs = Yabs = 0
    for i in range(len(X)-1): #last column is playlist_number
        Xabs += abs(X[i])
        Yabs += abs(Y[i])
        XY += X[i]*Y[i]
    return XY/(Xabs + Yabs)

In [89]:
def SimpleModel(X,Averages):
    best = 0.0
    for i in range(len(Averages)):
        if(CosSimularity(X,Averages[i]) > best):
            best = CosSimularity(X,Averages[i])
            playlist = Averages[i][7]
    return playlist

Testing the simple model

In [95]:
npData = np.array(data)
predictions = []
for i in npData:
        predictions.append(SimpleModel(i,AverageVals))

Using sklearn to compute metrics

In [96]:
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve

In [107]:
trainingdata = npData[:,7]
print(precision_score(npData[:,7], predictions,average="weighted"))

0.5434360157860083
